In [1]:
import tensorflow as tf

#“保存 TensorFlow 模型”这一短语通常表示保存以下两种元素之一：

#检查点，或
#SavedModel。
#检查点可以捕获模型使用的所有参数（tf.Variable 对象）的确切值。
#检查点不包含对模型所定义计算的任何描述，因此通常仅在将使用保存参数值的源代码可用时才有用。

#另一方面，除了参数值（检查点）之外，SavedModel 格式还包括对模型所定义计算的序列化描述。
#这种格式的模型独立于创建模型的源代码。因此，它们适合通过 TensorFlow Serving、TensorFlow Lite、TensorFlow.js 
#或者使用其他编程语言（C、C++、Java、Go、Rust、C# 等 TensorFlow API）编写的程序进行部署。

#本文介绍用于编写和读取检查点的 API。

In [5]:
### 7.1.1 从 tf.keras 训练 API 保存

class Net(tf.keras.Model):
  """A simple linear model."""

  def __init__(self):
    super(Net, self).__init__()
    self.l1 = tf.keras.layers.Dense(5)

  def call(self, x):
    return self.l1(x)

In [6]:
net = Net()
net.save_weights('__Checkpoint/easy_checkpoint')

In [7]:
### 7.1.2 编写检查点
#为了帮助演示 tf.train.Checkpoint 的所有功能， 下面定义了一个玩具 (toy) 数据集和优化步骤：

##1.设置
def toy_dataset():
  inputs = tf.range(10.)[:, None]
  labels = inputs * 5. + tf.range(5.)[None, :]
  return tf.data.Dataset.from_tensor_slices(
    dict(x=inputs, y=labels)).repeat().batch(2)



In [8]:
def train_step(net, example, optimizer):
  """Trains `net` on `example` using `optimizer`."""
  with tf.GradientTape() as tape:
    output = net(example['x'])
    loss = tf.reduce_mean(tf.abs(output - example['y']))
  variables = net.trainable_variables
  gradients = tape.gradient(loss, variables)
  optimizer.apply_gradients(zip(gradients, variables))
  return loss

In [9]:
##2.创建检查点对象
#使用 tf.train.Checkpoint 对象手动创建一个检查点，其中要检查的对象设置为对象的特性。
#tf.train.CheckpointManager 也有助于管理多个检查点。
opt = tf.keras.optimizers.Adam(0.1)
dataset = toy_dataset()
iterator = iter(dataset)
ckpt = tf.train.Checkpoint(step=tf.Variable(1), optimizer=opt, net=net, iterator=iterator)
manager = tf.train.CheckpointManager(ckpt, '__Checkpoint/tf_ckpts', max_to_keep=3)

In [10]:
##3.训练模型并为模型设置检查点
#以下训练循环可创建模型和优化器的实例，然后将它们收集到 tf.train.Checkpoint 对象中。
#它在每批数据上循环调用训练步骤，并定期将检查点写入磁盘。

def train_and_checkpoint(net, manager):
  ckpt.restore(manager.latest_checkpoint)
  if manager.latest_checkpoint:
    print("Restored from {}".format(manager.latest_checkpoint))
  else:
    print("Initializing from scratch.")

  for _ in range(50):
    example = next(iterator)
    loss = train_step(net, example, opt)
    ckpt.step.assign_add(1)
    if int(ckpt.step) % 10 == 0:
      save_path = manager.save()
      print("Saved checkpoint for step {}: {}".format(int(ckpt.step), save_path))
      print("loss {:1.2f}".format(loss.numpy()))

In [11]:
train_and_checkpoint(net, manager)

Initializing from scratch.
Saved checkpoint for step 10: __Checkpoint/tf_ckpts\ckpt-1
loss 27.70
Saved checkpoint for step 20: __Checkpoint/tf_ckpts\ckpt-2
loss 21.12
Saved checkpoint for step 30: __Checkpoint/tf_ckpts\ckpt-3
loss 14.57
Saved checkpoint for step 40: __Checkpoint/tf_ckpts\ckpt-4
loss 8.15
Saved checkpoint for step 50: __Checkpoint/tf_ckpts\ckpt-5
loss 3.02


In [12]:
##4.恢复和继续训练
#在第一个训练周期结束后，您可以传递一个新的模型和管理器，但在您中断的地方继续训练：
opt = tf.keras.optimizers.Adam(0.1)
net = Net()
dataset = toy_dataset()
iterator = iter(dataset)
ckpt = tf.train.Checkpoint(step=tf.Variable(1), optimizer=opt, net=net, iterator=iterator)
manager = tf.train.CheckpointManager(ckpt, '__Checkpoint/tf_ckpts', max_to_keep=3)

train_and_checkpoint(net, manager)

Restored from __Checkpoint/tf_ckpts\ckpt-5
Saved checkpoint for step 60: __Checkpoint/tf_ckpts\ckpt-6
loss 0.62
Saved checkpoint for step 70: __Checkpoint/tf_ckpts\ckpt-7
loss 0.50
Saved checkpoint for step 80: __Checkpoint/tf_ckpts\ckpt-8
loss 0.65
Saved checkpoint for step 90: __Checkpoint/tf_ckpts\ckpt-9
loss 0.26
Saved checkpoint for step 100: __Checkpoint/tf_ckpts\ckpt-10
loss 0.19


In [13]:
#tf.train.CheckpointManager 对象会删除旧的检查点。上面配置为仅保留最近的三个检查点。
manager.checkpoints

['__Checkpoint/tf_ckpts\\ckpt-8',
 '__Checkpoint/tf_ckpts\\ckpt-9',
 '__Checkpoint/tf_ckpts\\ckpt-10']

In [ ]:
#这些路径（如 './tf_ckpts/ckpt-10'）不是磁盘上的文件，而是一个 index 文件和一个或多个包含变量值的数据文件的前缀。
#这些前缀被分组到一个单独的 checkpoint 文件 ('./tf_ckpts/checkpoint') 中，其中 CheckpointManager 保存其状态。